Install 🤗 Transformers and 🤗 Datasets

In [1]:
! pip install datasets transformers[torch]
! pip install --upgrade transformers[torch]
! pip install causal-conv1d
! pip install mamba-ssm
! pip install --upgrade mamba-ssm
! pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 17.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64

In [2]:
from datasets import load_dataset, load_metric
# from peft import LoraConfig
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, MambaPreTrainedModel, MambaModel, PretrainedConfig, AutoConfig, MambaForCausalLM
from transformers.modeling_outputs import SequenceClassifierOutput
import torch
import torch.nn as nn
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
import os
import numpy as np
from typing import Any, Dict, Optional, Tuple, Union
import pickle
from sklearn.metrics import accuracy_score

In [3]:
from huggingface_hub import login, notebook_login

notebook_login()
# login('[YOUR HF API KEY]')

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
task = "sst2"
model_id = "state-spaces/mamba-130m-hf"
batch_size = 64
max_seq_length = 512

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [6]:
actual_task = "mnli" if task == "mnli-mm" else task

# if os.path.exists("dataset.pkl"):
#         print(f"Loading dataset from dataset.pkl")
#         with open("dataset.pkl", 'rb') as f:
#             dataset = pickle.load(f)
# else:
#     print(f"dataset.pkl not found. Loading dataset from original source and saving it.")
#     dataset = load_dataset("glue", actual_task)
#     with open("dataset.pkl", 'wb') as f:
#         pickle.dump(dataset, f)

dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)
metric_sklearn = lambda y_pred,y_true: accuracy_score(y_true,y_pred>0.5)
print(dataset,metric,sep='\n')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

<ipython-input-6-0dbf00d84697>:14: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', actual_task)
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})
Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Example

In [7]:
num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
# tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
  print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
  print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
  print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")
def preprocess_function(examples):
  if sentence2_key is None:
    return tokenizer(examples[sentence1_key], max_length=max_seq_length, truncation=True)
  return tokenizer(examples[sentence1_key], examples[sentence2_key], max_length=max_seq_length, truncation=True)

encoded_dataset = dataset.map(preprocess_function, batched=True)

Sentence: hide new secretions from the parental units 


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [53]:
! pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00


In [61]:
from transformers.models.mamba.modeling_mamba import *

def forward(
    self,
    input_ids: Optional[torch.LongTensor] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    cache_params: Optional[MambaCache] = None,
    labels: Optional[torch.LongTensor] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
    use_cache: Optional[bool] = None,
    **kwargs,  # for now we need this for generation
) -> Union[Tuple, MambaCausalLMOutput]:
    r"""
    labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
        Labels for language modeling. Note that the labels **are shifted** inside the model, i.e. you can set
        `labels = input_ids` Indices are selected in `[-100, 0, ..., config.vocab_size]` All labels set to `-100`
        are ignored (masked), the loss is only computed for labels in `[0, ..., config.vocab_size]`
    """
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict

    mamba_outputs = self.backbone(
        input_ids,
        cache_params=cache_params,
        inputs_embeds=inputs_embeds,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
        use_cache=use_cache,
    )
    hidden_states = mamba_outputs[0]

    logits = self.lm_head(hidden_states.to(self.lm_head.weight.dtype)).float()

    mask = kwargs.get("attention_mask", None)
    indices = (torch.sum(mask, dim=1, keepdim=True)-1).unsqueeze(-1)
    indices = indices.expand(logits.shape[0], 1, logits.shape[2])
    logits = torch.gather(logits, 1, indices).squeeze(1).contiguous()

    loss = None
    if labels is not None:
        # move labels to correct device to enable model parallelism
        labels = labels.to(logits.device)

        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits, labels)

    if not return_dict:
        output = (logits,) + mamba_outputs[1:]
        return ((loss,) + output) if loss is not None else output

    return MambaCausalLMOutput(
        loss=loss,
        logits=logits,
        cache_params=mamba_outputs.cache_params,
        hidden_states=mamba_outputs.hidden_states,
    )

MambaForCausalLM.forward = forward

def model_init():
    model = MambaForCausalLM.from_pretrained(model_id)
    model.config.keys_to_ignore_at_inference = ["cache_params", "hidden_states"]
    model.lm_head = torch.nn.Linear(model.config.d_model, num_labels)
    return model

In [62]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    # print(predictions)
    # print(labels)

    # print(metric.compute(predictions=predictions, references=labels))
    # print({'accuracy':metric_sklearn(predictions, labels)})

    # return metric.compute(predictions=predictions, references=labels)
    return {'accuracy':metric_sklearn(predictions, labels)}

validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"

class MambaTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    outputs = model(**inputs)
    lm_logits = outputs.logits
    lm_loss = outputs.loss

    if return_outputs:
      return lm_loss, outputs
    else:
      return lm_loss

  def save_model(self, output_dir, _internal_call):
    if not os.path.exists(output_dir):
      os.makedirs(output_dir)

    torch.save(self.model.state_dict(), f"{output_dir}/pytorch_model.bin")
    self.tokenizer.save_pretrained(output_dir)

In [63]:
from peft import LoraConfig, get_peft_model

lora_config =  LoraConfig(
        r=2,
        lora_alpha=4,
        target_modules=["x_proj", "embeddings", "in_proj", "out_proj"],
        task_type="SEQ_CLS",
        bias="none"
)

# model = get_peft_model(model_init(), lora_config)
# model.print_trainable_parameters()
model = model_init()

In [64]:
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit

# Soft Prompt
config = PromptTuningConfig(task_type=TaskType.SEQ_CLS, num_virtual_tokens=10, num_attention_heads=0)
config

PromptTuningConfig(peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.SEQ_CLS: 'SEQ_CLS'>, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=0, num_layers=None, prompt_tuning_init=<PromptTuningInit.RANDOM: 'RANDOM'>, prompt_tuning_init_text=None, tokenizer_name_or_path=None, tokenizer_kwargs=None)

In [40]:
from peft import LoraConfig, get_peft_model, PromptEncoderConfig, TaskType, PromptEncoderReparameterizationType

config = PromptEncoderConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=10,
    encoder_reparameterization_type=PromptEncoderReparameterizationType.MLP,
    encoder_dropout=0.1,
    encoder_num_layers=5,
    encoder_hidden_size=1024,
    num_attention_heads=0
)
config

PromptEncoderConfig(peft_type=<PeftType.P_TUNING: 'P_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.SEQ_CLS: 'SEQ_CLS'>, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=0, num_layers=None, encoder_reparameterization_type=<PromptEncoderReparameterizationType.MLP: 'MLP'>, encoder_hidden_size=1024, encoder_num_layers=5, encoder_dropout=0.1)

In [65]:
model = get_peft_model(model, config)

In [66]:
model.print_trainable_parameters()

trainable params: 7,680 || all params: 129,144,578 || trainable%: 0.005946823412129621


In [67]:
# Training once by arguments
torch.manual_seed(123)

args = TrainingArguments(
    f"Mamba-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1.1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    warmup_ratio=0.1,
    # max_steps=1
)

trainer = MambaTrainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # model_init=model,
)

trainer.train()

RuntimeError: gather(): Expected dtype int64 for index

In [ ]:
trainer.evaluate()["eval_" + metric_name]